In [ ]:
#=============================================================================
# Solar Farm Semantic Segmentation with U-Net - Reed Johnson - Mask/Tile Pairs 
# -----------------------------------------------------------------------------

# -------------------- Import Libraries --------------------
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# -------------------- Set Paths --------------------
# Input image tiles, ground truth masks, and predicted masks
image_dir = "/workspace/data_to_runpod/tiles_1024ft_v2"
mask_dir = "/workspace/data_to_runpod/masks_1024ft_v2"
pred_dir = "/workspace/outputs"


In [ ]:
# Output folder for saving overlay visualizations
output_dir = "/workspace/presentation_overlays"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# -------------------- Collect Files --------------------
# Get list of image files that have corresponding prediction masks
image_files = sorted([
    f for f in os.listdir(image_dir)
    if f.endswith(".png") and os.path.exists(os.path.join(pred_dir, f.replace(".png", "_pred.png")))
])

In [ ]:
# ----------------- Define Loaders --------------------
def load_image(path):
    # Load RGB image as NumPy array
    return np.array(Image.open(path).convert("RGB"))

def load_mask(path):
    # Load mask, convert to binary mask
    mask = np.array(Image.open(path).convert("L"))
    return (mask == 255).astype(np.uint8)

# ---------------- Define Overlay Function ----------------
def overlay_mask(image, mask, color=(255, 0, 255), alpha=0.5):
    overlay = image.copy()
    color = np.array(color).reshape(1, 1, 3)
    mask_expanded = np.expand_dims(mask, axis=-1)
    overlay = np.where(mask_expanded, (1 - alpha) * image + alpha * color, image).astype(np.uint8)
    return overlay

# --------------------Define Save Overlay --------------------
def save_overlay(image, gt_mask, pred_mask, title, out_path):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    overlay_gt = overlay_mask(image, gt_mask, color=(128, 0, 128), alpha=0.5)   # Purple: Ground Truth Mask
    overlay_pred = overlay_mask(image, pred_mask, color=(255, 255, 0), alpha=0.5)  # Yellow: Prediction Mask

    axs[0].imshow(overlay_gt)
    axs[0].set_title("Ground Truth")
    axs[0].axis("off")

    axs[1].imshow(overlay_pred)
    axs[1].set_title("Prediction")
    axs[1].axis("off")

    fig.suptitle(title, fontsize=12)
    plt.tight_layout()
    fig.savefig(out_path, dpi=150)
    plt.close(fig)


In [ ]:
# -------------------- Generate PNGs --------------------
# Loop through matched image files and generate overlay comparisons
for fname in image_files[:100]:
    image_path = os.path.join(image_dir, fname)
    mask_path = os.path.join(mask_dir, fname.replace("tile_", "mask_"))
    pred_path = os.path.join(pred_dir, fname.replace(".png", "_pred.png"))

    image = load_image(image_path)
    mask = load_mask(mask_path)
    pred = load_mask(pred_path)

    out_file = os.path.join(output_dir, fname.replace(".png", "_overlay.png"))
    save_overlay(image, mask, pred, title=fname, out_path=out_file)